# Analyzing Streaming Service Content in SQL

Welcome to your webinar workspace! You can follow along as we analyze the data in a SQL database and visualize the results.

To set up a connection to the database:
- Select 'Databases' in the left hand sidebar,
- Click 'Connect Database',
- Select 'PostgreSQL' and fill in the connection details below.

Connection details:
- **Database connection name**: Streaming Codealong
- **Hostname**: [workspacedemodb.datacamp.com](http://workspacedemodb.datacamp.com/)
- **Database**: streaming
- **Username**: streaming_codealong
- **Password**: streaming_codealong

To consult the solution, head over to the file browser and select `notebook-solution.ipynb`.

## Exploring our data
Let's start by checking out the data we will be working with. We can start with the `amazon`, `hulu`, `netflix`, and `disney` tables.

In [5]:
SELECT *
FROM amazon

,id,title,year,age,imdb,rotten_tomatoes,type
0,4928,Fist of Fury,1972,18+,None,69/100,0
1,6813,Bazodee,2016,13+,None,47/100,0
2,4929,Fanboys,2009,13+,None,69/100,0
3,3784,Underworld: Evolution,2006,18+,None,76/100,0
4,3788,Across the Universe,2007,13+,None,76/100,0
...,...,...,...,...,...,...,...
2952,2104,Elfen Lied,2004,18+,8.0/10,77/100,1
2953,2117,Stargate Atlantis,2004,7+,8.1/10,76/100,1
2954,2301,Ghost Whisperer,2005,7+,6.4/10,68/100,1
2955,2289,The Amazing Race,2001,7+,7.6/10,68/100,1


In [6]:
SELECT *
FROM hulu

,id,title,year,age,imdb,rotten_tomatoes,type
0,3863,Bound,1996,18+,None,72/100,0
1,1971,Firefly,2002,16+,9.0/10,90/100,1
2,4091,Brian Banks,2019,13+,None,63/100,0
3,4083,Assassination Nation,2018,18+,None,64/100,0
4,4092,Soul Food,1997,18+,None,63/100,0
...,...,...,...,...,...,...,...
1670,2545,Whose Line Is It Anyway? (UK),1988,7+,8.2/10,60/100,1
1671,2544,The Incredible Dr. Pol,2011,7+,8.6/10,60/100,1
1672,2541,The Real Housewives of Beverly Hills,2010,16+,5.1/10,60/100,1
1673,2540,Dharma & Greg,1997,16+,6.3/10,60/100,1


In [7]:
SELECT *
FROM netflix

,id,title,year,age,imdb,rotten_tomatoes,type
0,3606,Match,2017,None,None,32/100,0
1,2561,The Binding,2020,None,None,47/100,0
2,2215,ATM,2012,None,None,50/100,0
3,3220,Look for a Star,2009,None,None,40/100,0
4,3605,Arango y Sanint: Ríase el show,2018,None,None,32/100,0
...,...,...,...,...,...,...,...
4743,1945,Million Yen Women,2017,None,None,14/100,1
4744,1955,Paul Hollywood's Big Continental Road Trip,2017,None,None,13/100,1
4745,2716,Paradox,2009,16+,7.0/10,56/100,1
4746,2637,Midnight Sun,2016,None,7.5/10,58/100,1


In [8]:
SELECT *
FROM disney

,id,title,year,age,imdb,rotten_tomatoes,type
0,4132,The Kid,2019,18+,None,62/100,0
1,9198,Hatching Pete,2009,all,None,52/100,0
2,9200,Chef Donald,1941,None,None,52/100,0
3,9201,Tiger Cruise,2005,7+,None,52/100,0
4,9204,Amy,1981,all,None,52/100,0
...,...,...,...,...,...,...,...
779,5521,Sydney to the Max,2019,all,6.3/10,52/100,1
780,5518,Silver Surfer,1998,7+,7.1/10,52/100,1
781,5558,The Mickey Mouse Club,1955,all,7.7/10,48/100,1
782,2152,Genius,2017,16+,8.3/10,74/100,1


We can also inspect the `genres` table, which is different from the other tables.

In [9]:
SELECT *
FROM genres

,film,genre
0,Sara's Notebook,"Dramas, International Movies, Thrillers"
1,Rare Exports: A Christmas Tale,"Action, Adventure, Comedy"
2,Gretel & Hansel,"Drama, Horror, Mystery"
3,Mr. Jones,"Drama, History, Thriller"
4,The Limehouse Golem,"Crime, Mystery, Thriller"
...,...,...
9590,The Incredible Dr. Pol,"Documentary, Reality-TV"
9591,The Real Housewives of Beverly Hills,Reality
9592,Dharma & Greg,"Comedy, Sitcom"
9593,Mobile Suit Gundam Wing,"Anime, International, Science Fiction"


## Preparing our data
### Joining the different tables
Our data appears to mostly have the same column names. So we can join the data with a series of [`UNION`](https://www.postgresql.org/docs/14/queries-union.html)s, which will append each table to the previous one.

We use `UNION ALL` to preserve any possible duplicate rows, as we will want to count entries if they appear in multiple services.

In [10]:
SELECT *, 'amazon' AS service
FROM amazon
UNION ALL
SELECT *, 'hulu' AS service
FROM hulu
UNION ALL
SELECT *, 'netflix' AS service
FROM netflix
UNION ALL
SELECT *, 'disney' AS service
FROM disney

,id,title,year,age,imdb,rotten_tomatoes,type,service
0,4928,Fist of Fury,1972,18+,None,69/100,0,amazon
1,6813,Bazodee,2016,13+,None,47/100,0,amazon
2,4929,Fanboys,2009,13+,None,69/100,0,amazon
3,3784,Underworld: Evolution,2006,18+,None,76/100,0,amazon
4,3788,Across the Universe,2007,13+,None,76/100,0,amazon
...,...,...,...,...,...,...,...,...
10159,5521,Sydney to the Max,2019,all,6.3/10,52/100,1,disney
10160,5518,Silver Surfer,1998,7+,7.1/10,52/100,1,disney
10161,5558,The Mickey Mouse Club,1955,all,7.7/10,48/100,1,disney
10162,2152,Genius,2017,16+,8.3/10,74/100,1,disney


One problem with the above approach is that we lose out on the streaming service information. So let's repeat our query, but add in the required info!

In [11]:
SELECT *, 'amazon' AS service
FROM amazon
UNION ALL
SELECT *, 'hulu' AS service
FROM hulu
UNION ALL
SELECT *, 'netflix' AS service
FROM netflix
UNION ALL
SELECT *, 'disney' AS service
FROM disney

,id,title,year,age,imdb,rotten_tomatoes,type,service
0,4928,Fist of Fury,1972,18+,None,69/100,0,amazon
1,6813,Bazodee,2016,13+,None,47/100,0,amazon
2,4929,Fanboys,2009,13+,None,69/100,0,amazon
3,3784,Underworld: Evolution,2006,18+,None,76/100,0,amazon
4,3788,Across the Universe,2007,13+,None,76/100,0,amazon
...,...,...,...,...,...,...,...,...
10159,5521,Sydney to the Max,2019,all,6.3/10,52/100,1,disney
10160,5518,Silver Surfer,1998,7+,7.1/10,52/100,1,disney
10161,5558,The Mickey Mouse Club,1955,all,7.7/10,48/100,1,disney
10162,2152,Genius,2017,16+,8.3/10,74/100,1,disney


Great! But we have one more table that might prove useful. Let's add in the genre information with a join.

To do this, we will need to use a [Common Table Expression](https://www.postgresql.org/docs/current/queries-with.html), or CTE.

In [12]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
FROM amazon
UNION
SELECT *, 'hulu' AS service
FROM hulu
UNION
SELECT *, 'netflix' AS service
FROM netflix
UNION
SELECT *, 'disney' AS service
FROM disney
)

SELECT *
FROM service_data AS sd
LEFT JOIN genres AS g
	ON sd.title = g.film

,id,title,year,age,imdb,rotten_tomatoes,type,service,film,genre
0,4536,The Current Occupant,2020,None,None,49/100,0,hulu,The Current Occupant,Horror
1,751,Sofia the First,2013,all,6.9/10,57/100,1,netflix,Sofia the First,Kids' TV
2,1182,Donald Glover: Weirdo,2012,None,None,60/100,0,netflix,Donald Glover: Weirdo,Stand-Up Comedy
3,4543,Shangri-La Suite,2016,16+,None,49/100,0,hulu,Shangri-La Suite,"Comedy, Crime, Drama"
4,874,Intersection,2016,None,6.6/10,54/100,1,netflix,Intersection,"Drama, Suspense"
...,...,...,...,...,...,...,...,...,...,...
10159,1833,The Lovers,2017,18+,None,54/100,0,netflix,The Lovers,"Comedies, Dramas, Independent Movies"
10160,5613,Slackers,2002,18+,None,58/100,0,amazon,Slackers,Comedy
10161,287,A Futile and Stupid Gesture,2018,18+,None,75/100,0,netflix,A Futile and Stupid Gesture,Comedies
10162,97,Monty Python's Flying Circus,1969,16+,8.8/10,81/100,1,netflix,Monty Python's Flying Circus,"British TV Shows, Classic & Cult TV, Internati..."


### Inspecting missing data
It looks like we are missing some values in the `age` and `imdb` columns. We will also check the `rotten_tomatoes` column because we may use it later. Let's see how extensive this problem is.

To calculate the null values per column, we will use a combination of [SUM()](https://www.postgresql.org/docs/8.2/functions-aggregate.html) and [CASE WHEN](postgresql.org/docs/current/functions-conditional.html) to count the number of null values.

In [13]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
	FROM amazon
	UNION
	SELECT *, 'hulu' AS service
	FROM hulu
	UNION
	SELECT *, 'netflix' AS service
	FROM netflix
	UNION
	SELECT *, 'disney' AS service
	FROM disney
),

all_data AS (
	SELECT *
	FROM service_data AS sd
	LEFT JOIN genres AS g
		ON sd.title = g.film
)

SELECT
	SUM(CASE WHEN imdb IS NULL THEN 1 ELSE 0 END) AS imdb_missing,
	SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS age_missing,
	SUM(CASE WHEN rotten_tomatoes IS NULL THEN 1 ELSE 0 END) AS rt_missing
FROM all_data

,imdb_missing,age_missing,rt_missing
0,6901,3633,7


## Analyzing our data
### Which is the most family-friendly streaming service?
Let's start by looking at the most family-friendly streaming service by the percentage of content geared towards children.

We have our primary genre column, but that could leave out some content. A better way may be to use [pattern matching](https://www.postgresql.org/docs/current/functions-matching.html) to find any references to "kids", "family", etc.

In [14]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
	FROM amazon
	UNION
	SELECT *, 'hulu' AS service
	FROM hulu
	UNION
	SELECT *, 'netflix' AS service
	FROM netflix
	UNION
	SELECT *, 'disney' AS service
	FROM disney
),

all_data AS (
	SELECT *
	FROM service_data AS sd
	LEFT JOIN genres AS g
		ON sd.title = g.film
)

SELECT *
FROM all_data
WHERE genre ILIKE '%kids%'
	OR genre ILIKE '%family%'
	OR genre ILIKE '%children%'

,id,title,year,age,imdb,rotten_tomatoes,type,service,film,genre
0,751,Sofia the First,2013,all,6.9/10,57/100,1,netflix,Sofia the First,Kids' TV
1,2703,A Fairly Odd Summer,2014,all,None,45/100,0,hulu,A Fairly Odd Summer,"Children & Family Movies, Comedies"
2,9266,Hawaiian Holiday,1937,None,None,49/100,0,disney,Hawaiian Holiday,"Animation, Short, Comedy, Family, Sport"
3,9196,All in a Nutshell,1949,None,None,52/100,0,disney,All in a Nutshell,"Family, Comedy, Animation, Short"
4,1559,Motown Magic,2018,all,7.7/10,41/100,1,netflix,Motown Magic,"Kids' TV, TV Sci-Fi & Fantasy"
...,...,...,...,...,...,...,...,...,...,...
1536,2171,For the Birds,2018,None,None,50/100,0,amazon,For the Birds,"Animation, Short, Comedy, Family"
1537,9096,Minutemen,2008,all,None,56/100,0,disney,Minutemen,"Adventure, Comedy, Family, Sci-Fi"
1538,9381,Yellowstone Cubs,1963,all,None,45/100,0,disney,Yellowstone Cubs,Family
1539,8703,Frozen II,2019,7+,None,78/100,0,disney,Frozen II,"Animation, Adventure, Comedy, Family, Fantasy,..."


Great! That seems to be working. Let's adapt our query and use `CASE WHEN` to perform an aggregation and see which platform has the highest percentage of family content.

In [15]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
	FROM amazon
	UNION
	SELECT *, 'hulu' AS service
	FROM hulu
	UNION
	SELECT *, 'netflix' AS service
	FROM netflix
	UNION
	SELECT *, 'disney' AS service
	FROM disney
),

all_data AS (
	SELECT *
	FROM service_data AS sd
	LEFT JOIN genres AS g
		ON sd.title = g.film
)

SELECT
	service,
	AVG(CASE WHEN genre ILIKE '%kids%'
		OR genre ILIKE '%family%'
		OR genre ILIKE '%children%' THEN 1.0000 ELSE 0.0000 END) * 100 AS pct_family
FROM all_data
GROUP BY service
ORDER BY pct_family DESC


,service,pct_family
0,disney,74.744898
1,netflix,11.057287
2,hulu,10.985075
3,amazon,8.319242


,service,pct_family
0,disney,74.744898
1,netflix,11.057287
2,hulu,10.985075
3,amazon,8.319242


### Which has the highest-rated content?
We also have access to information on the ratings of each piece of content in the `rotten_tomatoes` column. We use [`SPLIT_PART()`](https://www.postgresql.org/docs/9.1/functions-string.html) to extract the number from the column and then cast (`::`) the result as a numeric.

We will also further split the data by movie and tv shows and visualize the result in a grouped bar chart.

In [17]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
	FROM amazon
	UNION
	SELECT *, 'hulu' AS service
	FROM hulu
	UNION
	SELECT *, 'netflix' AS service
	FROM netflix
	UNION
	SELECT *, 'disney' AS service
	FROM disney
),

all_data AS (
	SELECT *
	FROM service_data AS sd
	LEFT JOIN genres AS g
		ON sd.title = g.film
)

SELECT
	service,
	CASE WHEN type = 1 THEN 'TV' ELSE 'Movie' END AS type,
	AVG(SPLIT_PART(rotten_tomatoes, '/', 1)::NUMERIC) AS rt_score
FROM all_data
GROUP BY service, type
ORDER BY service, type

,service,type,rt_score
0,amazon,Movie,51.990146
1,amazon,TV,52.377207
2,disney,Movie,60.047934
3,disney,TV,54.486034
4,hulu,Movie,60.482517
5,hulu,TV,59.690625
6,netflix,Movie,54.965913
7,netflix,TV,54.229586


,service,type,rt_score
0,amazon,Movie,51.990146
1,amazon,TV,52.377207
2,disney,Movie,60.047934
3,disney,TV,54.486034
4,hulu,Movie,60.482517
5,hulu,TV,59.690625
6,netflix,Movie,54.965913
7,netflix,TV,54.229586


### Have critics and audiences diverged over time?
Okay, for our final analysis, lets put the service aside and look into whether critics and audiences were more aligned on tv shows in the past.

To prepare the date for the chart cell, we will need to use [`TO_DATE()`](https://www.postgresql.org/docs/current/functions-formatting.html) to convert the year into a date.

In [24]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
	FROM amazon
	UNION
	SELECT *, 'hulu' AS service
	FROM hulu
	UNION
	SELECT *, 'netflix' AS service
	FROM netflix
	UNION
	SELECT *, 'disney' AS service
	FROM disney
),

all_data AS (
	SELECT *
	FROM service_data AS sd
	LEFT JOIN genres AS g
		ON sd.title = g.film
)

SELECT
	date,
	AVG(ABS(imdb_score - rt_score)) AS avg_difference
FROM (
	SELECT
		TO_DATE(year::TEXT, 'YYYY') AS date,
		SPLIT_PART(rotten_tomatoes, '/', 1)::NUMERIC AS rt_score,
		SPLIT_PART(imdb, '/', 1)::NUMERIC * 10 AS imdb_score
	FROM all_data
	WHERE imdb IS NOT NULL
		AND rotten_tomatoes IS NOT NULL
		AND year >= 2000
	) AS sub
GROUP BY date
ORDER BY date

,date,avg_difference
0,2000-01-01 00:00:00+00:00,12.857143
1,2001-01-01 00:00:00+00:00,10.941176
2,2002-01-01 00:00:00+00:00,12.526316
3,2003-01-01 00:00:00+00:00,11.103448
4,2004-01-01 00:00:00+00:00,10.741935
5,2005-01-01 00:00:00+00:00,10.962264
6,2006-01-01 00:00:00+00:00,12.092593
7,2007-01-01 00:00:00+00:00,13.825000
8,2008-01-01 00:00:00+00:00,13.155172
9,2009-01-01 00:00:00+00:00,11.759036


,date,avg_difference
0,2000-01-01 00:00:00+00:00,12.857143
1,2001-01-01 00:00:00+00:00,10.941176
2,2002-01-01 00:00:00+00:00,12.526316
3,2003-01-01 00:00:00+00:00,11.103448
4,2004-01-01 00:00:00+00:00,10.741935
5,2005-01-01 00:00:00+00:00,10.962264
6,2006-01-01 00:00:00+00:00,12.092593
7,2007-01-01 00:00:00+00:00,13.825000
8,2008-01-01 00:00:00+00:00,13.155172
9,2009-01-01 00:00:00+00:00,11.759036


What are the most divisive shows?

In [26]:
WITH service_data AS (
	SELECT *, 'amazon' AS service
	FROM amazon
	UNION
	SELECT *, 'hulu' AS service
	FROM hulu
	UNION
	SELECT *, 'netflix' AS service
	FROM netflix
	UNION
	SELECT *, 'disney' AS service
	FROM disney
),

all_data AS (
	SELECT *
	FROM service_data AS sd
	LEFT JOIN genres AS g
		ON sd.title = g.film
)

SELECT
	title,
	AVG(ABS(imdb_score - rt_score)) AS avg_difference
FROM (
	SELECT
		title,
		SPLIT_PART(rotten_tomatoes, '/', 1)::NUMERIC AS rt_score,
		SPLIT_PART(imdb, '/', 1)::NUMERIC * 10 AS imdb_score
	FROM all_data
	WHERE imdb IS NOT NULL
		AND rotten_tomatoes IS NOT NULL
		AND year >= 2000
	) AS sub
GROUP BY title
ORDER BY avg_difference DESC

,title,avg_difference
0,Chiquis 'n Control,52.0
1,Design Squad,51.0
2,Owlegories,51.0
3,Engineering Giants,51.0
4,On the Real,51.0
...,...,...
2752,Archer,0.0
2753,Preacher,0.0
2754,All About the Washingtons,0.0
2755,Fuller House,0.0
